# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head(5)

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
total_qty_customer = data.groupby(by=['CustomerID', 'ProductName'])[['Quantity']].sum().reset_index()

total_qty_customer.head(5)

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
pvt_product=data.pivot_table(values='Quantity', columns='CustomerID', index='ProductName', aggfunc='sum').fillna(0)
pvt_product.head(5)


CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
similarity_matrix= squareform(pdist(pvt_product.T, 'euclidean'))

similarity_matrix

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

In [7]:
distances = pd.DataFrame(1/(1 + squareform(pdist(pvt_product.T, 'euclidean'))),index=pvt_product.columns, columns=pvt_product.columns)

distances.head(5)

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [8]:
customer33 = distances[33].sort_values(ascending=False)

top_5= customer33[1:6].index.to_list()

top_5

[264, 3535, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [9]:
total_qty_customer.index[0:0]

RangeIndex(start=0, stop=0, step=1)

In [10]:
total_qty_customer_top5 = total_qty_customer[total_qty_customer['CustomerID'].isin(top_5)]

total_qty_customer_top5.head(5)

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [11]:
total_product = total_qty_customer_top5.groupby('ProductName')[['Quantity']].sum().sort_values('Quantity',ascending=False)

total_product.head(5)

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [12]:
no_purchase = pvt_product[pvt_product[33]==0]
products_no_purchase = no_purchase.merge(total_product, left_on='ProductName', right_on='ProductName').sort_values('Quantity',ascending=False)
products_no_purchase[1:6].index.to_list()

['Soup - Campbells Bean Medley',
 'Wine - Ej Gallo Sierra Valley',
 'Butter - Unsalted',
 'Yoghurt Tubes',
 'Cod - Black Whole Fillet']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [13]:
recomendation = {}
keys = data['CustomerID'].unique()
keys = keys.tolist()

In [14]:
for key in keys:
    distancia_customer = distances[key].sort_values(ascending=False)
    top5_customer = distancia_customer[1:6].index.to_list()
    similar_customers = total_qty_customer[total_qty_customer['CustomerID'].isin(top5_customer)]
    product_total = similar_customers.groupby('ProductName')[['Quantity']].sum().sort_values('Quantity',ascending=False)
    no_purchase_product = pvt_product[pvt_product[key]==0]
    final_list = no_purchase_product.merge(product_total, left_on='ProductName', right_on='ProductName').sort_values('Quantity',ascending=False)
    final = final_list[1:6].index
    recomendation.update({key:final})

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [15]:
recommender = pd.DataFrame(recomendation)
recommender = recommender.T
recommender.columns=['Product_1','Product_2','Product_3','Product_4','Product_5']
recommender.index.names=['CustomerID']
recommender.head(5)

,Product_1,Product_2,Product_3,Product_4,Product_5
CustomerID,,,,,
61288,Jagermeister,Lime Cordial - Roses,Macaroons - Two Bite Choc,"Water - Mineral, Natural",Flavouring - Orange
77352,Tahini Paste,Tuna - Salad Premix,Chicken - Soup Base,Isomalt,Guinea Fowl
40094,Puree - Mocha,Tuna - Salad Premix,Sherry - Dry,Beer - Sleemans Cream Ale,"Oregano - Dry, Rubbed"
23548,Pernod,Wanton Wrap,Bread - Raisin Walnut Oval,Banana Turning,Sprouts - Baby Pea Tendrils
78981,Lettuce - Frisee,"Yogurt - Blueberry, 175 Gr",Cinnamon Buns Sticky,Pop Shoppe Cream Soda,Beef - Inside Round


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [27]:
similarity_matrix_new= squareform(pdist(pvt_product.T, 'jaccard'))

similarity_matrix_new

array([[0.        , 0.94308943, 0.88785047, ..., 1.        , 1.        ,
        1.        ],
       [0.94308943, 0.        , 0.92741935, ..., 1.        , 1.        ,
        1.        ],
       [0.88785047, 0.92741935, 0.        , ..., 1.        , 1.        ,
        1.        ],
       ...,
       [1.        , 1.        , 1.        , ..., 0.        , 0.90551181,
        0.95081967],
       [1.        , 1.        , 1.        , ..., 0.90551181, 0.        ,
        0.9       ],
       [1.        , 1.        , 1.        , ..., 0.95081967, 0.9       ,
        0.        ]])

In [28]:
distances_new = pd.DataFrame(1/(1 + squareform(pdist(pvt_product.T, 'jaccard'))),index=pvt_product.columns, columns=pvt_product.columns)

distances_new.head(5)

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.514644,0.529703,0.524038,0.513043,0.516129,0.510549,0.509174,0.511211,0.513889,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
200,0.514644,1.000000,0.518828,0.514523,0.503704,0.512000,0.520325,0.510204,0.518828,0.519313,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
264,0.529703,0.518828,1.000000,0.515695,0.527027,0.515419,0.510460,0.508772,0.515419,0.511013,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
356,0.524038,0.514523,0.515695,1.000000,0.517094,0.510917,0.508403,0.521531,0.510823,0.513393,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
412,0.513043,0.503704,0.527027,0.517094,1.000000,0.512605,0.516529,0.520362,0.510373,0.515284,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5


In [29]:
recomendation_new = {}
keys = data['CustomerID'].unique()
keys = keys.tolist()

In [30]:
for key in keys:
    distancia_customer = distances_new[key].sort_values(ascending=False)
    top5_customer = distancia_customer[1:6].index.to_list()
    similar_customers = total_qty_customer[total_qty_customer['CustomerID'].isin(top5_customer)]
    product_total = similar_customers.groupby('ProductName')[['Quantity']].sum().sort_values('Quantity',ascending=False)
    no_purchase_product = pvt_product[pvt_product[key]==0]
    final_list = no_purchase_product.merge(product_total, left_on='ProductName', right_on='ProductName').sort_values('Quantity',ascending=False)
    final = final_list[1:6].index
    recomendation_new.update({key:final})

In [31]:
recommender_new = pd.DataFrame(recomendation_new)
recommender_new = recommender_new.T
recommender_new.columns=['Product_1','Product_2','Product_3','Product_4','Product_5']
recommender_new.index.names=['CustomerID']
recommender_new.head(5)

,Product_1,Product_2,Product_3,Product_4,Product_5
CustomerID,,,,,
61288,Flavouring - Orange,V8 - Berry Blend,Ecolab - Mikroklene 4/4 L,General Purpose Trigger,Cornflakes
77352,Sea Bass - Whole,Beer - Blue,Cattail Hearts,Mayonnaise - Individual Pkg,Beef Ground Medium
40094,Rosemary - Dry,"Juice - Cranberry, 341 Ml","Wine - Red, Colio Cabernet",Pasta - Angel Hair,Snapple - Iced Tea Peach
23548,Table Cloth 81x81 White,Longos - Chicken Wings,Potatoes - Idaho 100 Count,"Lemonade - Natural, 591 Ml","Ice - Clear, 300 Lb For Carving"
78981,Wine - Blue Nun Qualitatswein,Pie Filling - Cherry,Cocoa Butter,Cheese - Mix,Beer - Sleemans Cream Ale
